# 사용 라이브러리 설치 및 import

In [5]:
!pip install transformers
!pip install sentencepiece
!pip install konlpy

     |████████████████████████████████| 3.8 MB 5.2 MB/s 
     |████████████████████████████████| 895 kB 43.7 MB/s 
     |████████████████████████████████| 596 kB 55.0 MB/s 
     |████████████████████████████████| 6.5 MB 38.1 MB/s 
     |████████████████████████████████| 67 kB 4.8 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
ERROR: Could not find a version that satisfies the requirement sentencpiece (from versions: none)
ERROR: No matching distribution found for sentencpiece
     |████████████████████████████████| 19.4 MB 1.2 MB/s 
     |████████████████████████████████| 448 kB 39.7 MB/s 


In [7]:
import transformers
transformers.__version__

'4.17.0'

In [8]:
import tensorflow as tf
import os 
import sys
import pprint
import json
import pandas as pd
import numpy as np
import urllib.request
from tqdm import tqdm
from transformers import BertTokenizer, TFBertModel

# 구글 드라이브, 코랩, GCP, TPU 환경 세팅

In [1]:
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse

Selecting previously unselected package google-drive-ocamlfuse.
(Reading database ... 155335 files and directories currently installed.)
Preparing to unpack .../google-drive-ocamlfuse_0.7.27-0ubuntu1~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.27-0ubuntu1~ubuntu18.04.1) ...
Setting up google-drive-ocamlfuse (0.7.27-0ubuntu1~ubuntu18.04.1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...


In [2]:
# google drive GCP 연동
from google.colab import auth
auth.authenticate_user()

from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()

import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()

!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
Please enter the verification code: Access token retrieved correctly.


In [3]:
# Gdrice 폴더 및 colab 연동
!mkdir -p Gdrive

!google-drive-ocamlfuse Gdrive -o nonempty

!ls

adc.json  Gdrive  sample_data


# tpu 설정
- tensorflow version 2.x
- 참고 
상세 문서 : https://www.tensorflow.org/guide/tpu
코랩 가이드 : 

In [9]:
from google.colab import auth
auth.authenticate_user()

resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='')
print('Running on TPU ', resolver.cluster_spec().as_dict()['worker'])

tf.config.experimental_connect_to_cluster(resolver)

# This is the TPU initialization code that has to be at the beginning.
tf.tpu.experimental.initialize_tpu_system(resolver)
print("All devices: ", tf.config.list_logical_devices('TPU'))

Running on TPU  ['10.84.20.234:8470']
INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Initializing the TPU system: grpc://10.84.20.234:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.84.20.234:8470


INFO:tensorflow:Finished initializing TPU system.


INFO:tensorflow:Finished initializing TPU system.


All devices:  [LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:0', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:1', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:2', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:3', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:4', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:5', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:6', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:7', device_type='TPU')]


In [10]:
TPU_IP = resolver.cluster_spec().as_dict()['worker'][0]
print('Running on TPU IP:', TPU_IP)

Running on TPU IP: 10.84.20.234:8470


In [11]:
TPU_ADDRESS = 'grpc://'+os.environ['COLAB_TPU_ADDR']
TPU_ADDRESS

'grpc://10.84.20.234:8470'

In [12]:
# 여러 TPU에서 모델 실행하기
# 모델을 여러 TPU에 배포하기 위한 배포 전략
# 배포 전략 자세히 확인 링크 : https://www.tensorflow.org/guide/distributed_training
strategy = tf.distribute.TPUStrategy(resolver)
print("REPLICAS: ", strategy.num_replicas_in_sync)

INFO:tensorflow:Found TPU system:


INFO:tensorflow:Found TPU system:


INFO:tensorflow:*** Num TPU Cores: 8


INFO:tensorflow:*** Num TPU Cores: 8


INFO:tensorflow:*** Num TPU Workers: 1


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


REPLICAS:  8


- 이 아래 작업은 tf version 1.x 에서 사용했는데 필요 없을수도 있음

In [ ]:
TPU_ADDRESS = 'grpc://'+os.environ['COLAB_TPU_ADDR']
TPU_ADDRESS

In [ ]:
# TPU address 확인
assert 'COLAB_TPU_ADDR' in os.environ, 'ERROR: Not connected to a TPU runtime; please see the first cell in this notebook for instructions!'

# TPU 주소 추출
TPU_ADDRESS = 'grpc://'+os.environ['COLAB_TPU_ADDR']
print('TPU address is', TPU_ADDRESS)

# 코랩의 인증 모듈로 할당된 TPU 모델 확인하기
from google.colab import auth
auth.authenticate_user()

# 
with tf.compat.v1.Session(TPU_ADDRESS) as session:
    print('TPU devices:')
    pprint.pprint(session.list_devices())

    # TPU에 credentials 정보 저장하기
    with open('/content/adc.json', 'r') as f :
        auth_info = json.load(f)
    
    # google cloud storage(bucket)에 현재 TPU와 인증 정보 설정하기
    # 이 세팅을 해야 gcp bucket과 연동 가능
    #tf.contrib.cloud.configure_gcs(session, credentials=auth_info)

# 학습 데이터 어떻게 준비해야 하는지 확인하기 위한 Sample

- 이진분류 네이버 영화 리뷰 텍스트

In [ ]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt", filename="ratings_train.txt")
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt", filename="ratings_test.txt")

train_data = pd.read_table('ratings_train.txt')
test_data = pd.read_table('ratings_test.txt')

print('훈련용 데이터 개수 :', len(train_data))
print('테스트용 데이터 개수 :', len(test_data))

훈련용 데이터 개수 : 150000
테스트용 데이터 개수 : 50000


In [ ]:
# 상위 5개 출력
train_data[:5]

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


In [ ]:
# 상위 5개 출력
test_data[:5]

,id,document,label
0,6270596,굳 ㅋ,1
1,9274899,GDNTOPCLASSINTHECLUB,0
2,8544678,뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아,0
3,6825595,지루하지는 않은데 완전 막장임... 돈주고 보기에는....,0
4,6723715,3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??,0


- 데이터 전처리

In [ ]:
train_data = train_data.dropna(how = 'any') # Null 값이 존재하는 행 제거
train_data = train_data.reset_index(drop=True)
print(train_data.isnull().values.any()) # Null 값이 존재하는지 확인

False


In [ ]:
test_data = test_data.dropna(how = 'any') # Null 값이 존재하는 행 제거
test_data = test_data.reset_index(drop=True)
print(test_data.isnull().values.any()) # Null 값이 존재하는지 확인

False


In [ ]:
# 전처리 후 남은 데이터 개수
print('train 데이터 개수 :',len(train_data))
print('test 데이터 개수 :',len(test_data))

train 데이터 개수 : 149995
test 데이터 개수 : 49997


- tokenizer 로드

In [ ]:
tokenizer = BertTokenizer.from_pretrained('klue/bert-base')

Downloading:   0%|          | 0.00/243k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/125 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/289 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/425 [00:00<?, ?B/s]

In [ ]:
# encoding sample
print(tokenizer.encode('이게 맞는건지 잘 모르겠어요 한국어 BERT가 확실한가?'))

[2, 4380, 1047, 2259, 2332, 2118, 1521, 3926, 2918, 10283, 9187, 38, 10646, 2081, 2116, 4592, 2470, 2116, 35, 3]


In [ ]:
# tokenizing
print(tokenizer.tokenize('이게 맞는건지 잘 모르겠어요 한국어 BERT가 확실한가?'))

['이게', '맞', '##는', '##건', '##지', '잘', '모르', '##겠', '##어요', '한국어', 'B', '##ER', '##T', '##가', '확실', '##한', '##가', '?']


In [ ]:
# decoding sample
print(tokenizer.decode(tokenizer.encode('이게 맞는건지 잘 모르겠어요 한국어 BERT가 확실한가?')))

[CLS] 이게 맞는건지 잘 모르겠어요 한국어 BERT가 확실한가? [SEP]


In [ ]:
# element 추출
for element in tokenizer.encode('이게 맞는건지 잘 모르겠어요 한국어 BERT가 확실한가?'):
    print(tokenizer.decode(element))

[ C L S ]
이 게
맞
# # 는
# # 건
# # 지
잘
모 르
# # 겠
# # 어 요
한 국 어
B
# # E R
# # T
# # 가
확 실
# # 한
# # 가
?
[ S E P ]


In [ ]:
print(tokenizer.pad_token, ':', tokenizer.pad_token_id)
print(tokenizer.unk_token, ':', tokenizer.unk_token_id)
print(tokenizer.cls_token, ':', tokenizer.cls_token_id)
print(tokenizer.sep_token, ':', tokenizer.sep_token_id)

[PAD] : 0
[UNK] : 1
[CLS] : 2
[SEP] : 3


# 한국어 BERT용 input 데이터 생성

In [ ]:
MAX_SEQ_LEN = 128

sample_encode = tokenizer.encode('이게 맞는건지 잘 모르겠어요 한국어 BERT가 확실한가?', 
                                 max_length=MAX_SEQ_LEN, padding='max_length')

print(sample_encode)
print('길이 :', len(sample_encode))


[2, 4380, 1047, 2259, 2332, 2118, 1521, 3926, 2918, 10283, 9187, 38, 10646, 2081, 2116, 4592, 2470, 2116, 35, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
길이 : 128


In [ ]:
# segment input
print([0]* MAX_SEQ_LEN)

# mask input
valid_num = len(tokenizer.encode('이게 맞는건지 잘 모르겠어요 한국어 BERT가 확실한가?'))
print(valid_num * [1] + (MAX_SEQ_LEN - valid_num) * [0])

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [ ]:
# input data 만들기 함수
def convert_examples_to_features(examples, labels, max_seq_len, tokenizer):

    input_ids, attention_masks, token_type_ids, data_labels = [], [], [], []

    for example, label in tqdm(zip(examples, labels), total=len(examples)):
        input_id = tokenizer.encode(example, max_length=MAX_SEQ_LEN, pad_to_max_length=True)
        padding_count = input_id.count(tokenizer.pad_token_id)
        attention_mask = [1] * (MAX_SEQ_LEN - padding_count) + [0] * padding_count
        token_type_id = [0] * MAX_SEQ_LEN

        assert len(input_id) == MAX_SEQ_LEN, 'Error with input length {} vs {}'.format(len(input_id), MAX_SEQ_LEN)
        assert len(attention_mask) == MAX_SEQ_LEN, "Error with attention mask length {} vs {}".format(len(attention_mask), max_seq_len)
        assert len(token_type_id) == MAX_SEQ_LEN, "Error with token type length {} vs {}".format(len(token_type_id), max_seq_len)

        input_ids.append(input_id)
        attention_masks.append(attention_mask)
        token_type_ids.append(token_type_id)
        data_labels.append(label)

    # 다 리스트로 데이터를 받기 때문에 np.array
    input_ids = np.array(input_ids, dtype=int)
    attention_masks = np.array(attention_masks, dtype=int)
    token_type_ids = np.array(token_type_ids, dtype=int)

    # label 데이터는 시리즈 형식으로 받아서 np.asarray 사용
    data_labels = np.asarray(data_labels, dtype=np.int32)

    return (input_ids, attention_masks, token_type_ids), data_labels

In [ ]:
# 학습용 데이터
train_X, train_y = convert_examples_to_features(train_data['document'], train_data['label'], max_seq_len=MAX_SEQ_LEN, tokenizer=tokenizer)

  0%|          | 0/149995 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2277: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
100%|██████████| 149995/149995 [01:01<00:00, 2420.77it/s]


In [ ]:
# 평가용 데이터
test_X, test_y = convert_examples_to_features(test_data['document'], test_data['label'], max_seq_len=MAX_SEQ_LEN, tokenizer=tokenizer)

  0%|          | 0/49997 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2277: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
100%|██████████| 49997/49997 [00:26<00:00, 1903.50it/s]


In [ ]:
# 최대 길이: 128
input_id = train_X[0][0]
attention_mask = train_X[1][0]
token_type_id = train_X[2][0]
label = train_y[0]

print('단어에 대한 정수 인코딩 :',input_id)
print('어텐션 마스크 :',attention_mask)
print('세그먼트 인코딩 :',token_type_id)
print('각 인코딩의 길이 :', len(input_id))
print('정수 인코딩 복원 :',tokenizer.decode(input_id))
print('레이블 :',label)

단어에 대한 정수 인코딩 : [   2 1376  831 2604   18   18 4229 9801 2075 2203 2182 4243    3    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0]
어텐션 마스크 : [1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
세그먼트 인코딩 : [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 

In [ ]:
# 한국어 BERT model load
model = TFBertModel.from_pretrained('klue/bert-base', from_pt=True)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.weight', 'bert.embeddings.position_ids', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the 

In [ ]:
# output 뽑아내기
max_seq_len = 128

input_ids_layer = tf.keras.layers.Input(shape=(max_seq_len,), dtype=tf.int32)
attention_masks_layer = tf.keras.layers.Input(shape=(max_seq_len,), dtype=tf.int32)
token_type_ids_layer = tf.keras.layers.Input(shape=(max_seq_len,), dtype=tf.int32)

# outputs ==> ('last_hidden_state', 'pooler_output')
outputs = model([input_ids_layer, attention_masks_layer, token_type_ids_layer])

In [ ]:
outputs

TFBaseModelOutputWithPoolingAndCrossAttentions([('last_hidden_state',
                                                 <KerasTensor: shape=(None, 128, 768) dtype=float32 (created by layer 'tf_bert_model_1')>),
                                                ('pooler_output',
                                                 <KerasTensor: shape=(None, 768) dtype=float32 (created by layer 'tf_bert_model_1')>)])

In [ ]:
# last_hidden_state
outputs[0]

<KerasTensor: shape=(None, 128, 768) dtype=float32 (created by layer 'tf_bert_model_1')>

In [ ]:
# pooler_output
outputs[1]

<KerasTensor: shape=(None, 768) dtype=float32 (created by layer 'tf_bert_model_1')>

In [ ]:
# BERT 분류 모델

class TFBertForSequenceClassification(tf.keras.Model):
    def __init__(self, model_name):
        super(TFBertForSequenceClassification, self).__init__()
        self.bert = TFBertModel.from_pretrained(model_name, from_pt=True)

        # ==> 이진분류
        self.classifier = tf.keras.layers.Dense(1,
                                                kernel_initializer=tf.keras.initializers.TruncatedNormal(0.02),
                                                activation='sigmoid',
                                                name='classifier')

    def call(self, inputs):
        input_ids, attention_mask, token_type_ids = inputs
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        cls_token = outputs[1] # ==> pooler_output
        prediction = self.classifier(cls_token)

        return prediction

In [ ]:
# TPU로 학습시키기
with strategy.scope():
  model = TFBertForSequenceClassification("klue/bert-base")
  optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
  loss = tf.keras.losses.BinaryCrossentropy()
  model.compile(optimizer=optimizer, loss=loss, metrics = ['accuracy'])

history = model.fit(train_X, train_y, epochs=2, batch_size=64, validation_split=0.2)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.weight', 'bert.embeddings.position_ids', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the 

Epoch 1/2
1875/1875 [==============================] - 308s 114ms/step - loss: 0.2835 - accuracy: 0.8798 - val_loss: 0.2380 - val_accuracy: 0.9029
Epoch 2/2
1875/1875 [==============================] - 197s 105ms/step - loss: 0.1887 - accuracy: 0.9258 - val_loss: 0.2363 - val_accuracy: 0.9059


In [ ]:
# 평가용 데이터 정확도
results = model.evaluate(test_X, test_y, batch_size=1024)
print("test loss, test acc: ", results)

49/49 [==============================] - 29s 400ms/step - loss: 0.2416 - accuracy: 0.9012
test loss, test acc:  [0.24161489307880402, 0.9012340903282166]


In [ ]:
# 예측 함수
def sentiment_predict(new_sentence):
    input_id = tokenizer.encode(new_sentence, max_length=max_seq_len, pad_to_max_length=True)

    padding_count = input_id.count(tokenizer.pad_token_id)
    attention_mask = [1] * (max_seq_len - padding_count) + [0] * padding_count
    token_type_id = [0] * max_seq_len

    input_ids = np.array([input_id])
    attention_masks = np.array([attention_mask])
    token_type_ids = np.array([token_type_id])

    encoded_input = [input_ids, attention_masks, token_type_ids]
    score = model.predict(encoded_input)[0][0]

    if(score > 0.5):
        print("{:.2f}% 확률로 긍정 리뷰입니다.\n".format(score * 100))
    else:
        print("{:.2f}% 확률로 부정 리뷰입니다.\n".format((1 - score) * 100))

In [ ]:
sentiment_predict('보던거라 계속보고있는데 전개도 느리고 주인공인 은희는 한두컷 나오면서 소극적인모습에 ')

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2277: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


98.67% 확률로 부정 리뷰입니다.

